In [1]:
# imports
import tensorflow as tf
import pandas as pd
import numpy as np
import io
import os
import time

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [2]:
df = pd.read_csv(r"C:\Users\chpav\OneDrive\Desktop\Dr reddy\brand_generator.csv")

In [3]:
df.head()

,Country,Drug name,Molecules name,Therapeutic 1,Therapeutic 2
0,ALGERIA,NATRIDENT,TOOTHPASTES,A1 STOMATOLOGICALS,A1A STOMATOLOGICALS
1,ALGERIA,NATRYL,MONOFLUOROPHOSPHORIC ACID,A1 STOMATOLOGICALS,A1A STOMATOLOGICALS
2,ALGERIA,SANITHOL,GERANIUM ROBERTIANUM!LEVOMENTHOL!SALICYLIC ACID,A1 STOMATOLOGICALS,A1A STOMATOLOGICALS
3,ALGERIA,GETRIL,HEXETIDINE,A1 STOMATOLOGICALS,A1A STOMATOLOGICALS
4,ALGERIA,DELABARRE,CROCUS SATIVUS!TAMARINDUS INDICA,A1 STOMATOLOGICALS,A1A STOMATOLOGICALS


In [8]:
df.describe()

,Country,Drug name
count,64818,64818
unique,75,39362
top,INDIA,VOLTAREN
freq,4642,75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221516 entries, 0 to 221515
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Country         221516 non-null  object
 1   Drug name       221516 non-null  object
 2   Molecules name  221516 non-null  object
 3   Therapeutic 1   221516 non-null  object
 4   Therapeutic 2   221516 non-null  object
dtypes: object(5)
memory usage: 8.5+ MB


In [3]:
df = df.drop(['Therapeutic 1', 'Therapeutic 2', 'Molecules name'], axis=1)

In [4]:
df = df.drop_duplicates()

In [5]:
df.info()
df.describe()
df.isna().sum() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64818 entries, 0 to 221510
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    64818 non-null  object
 1   Drug name  64818 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


Country      0
Drug name    0
dtype: int64

In [5]:
df = df[df['Country']=='INDIA']

In [6]:
df= df.drop(['Country'], axis=1)

In [7]:
step_length = 1 # The step length we take to get our samples from our corpus
epochs = 20 # Number of times we train on our full data
batch_size = 64 # Data samples in each training step
latent_dim = 500 # Size of our LSTM
dropout_rate = 0.2 # Regularization with dropout
model_path = os.path.realpath('./brand_gen_modelepochs=20v2.h5') # Location for the model
load_model = False # Enable loading model from disk
store_model = True # Store model to disk after training
verbosity = 1 # Print result for each epoch
gen_amount = 10 # How many

In [8]:
input_values = []
for value in df['Drug name']:
  input_values.append(value)

In [9]:
concat_names = '\n'.join(input_values).lower()

specialChars = "%&()+-./0123456789>\_` " 
for specialChar in specialChars:
    concat_names = concat_names.replace(specialChar, "")


chars = sorted(list(set(concat_names)))
print(chars)
num_chars = len(chars)




char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

max_sequence_length = max([len(name) for name in input_values])

print('Total chars: {}'.format(num_chars))
print('Corpus length:', len(concat_names))
print('Number of names: ', len(input_values)) 
print('Longest name: ', max_sequence_length)

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total chars: 27
Corpus length: 41547
Number of names:  4642
Longest name:  18


In [15]:
print(concat_names)

domeric
hydent
hydentk
kenacort
orahex
orahexpro
orahexplus
pediaflor
toothmin
g
orazinc
orogard
aqubenz
wetfast
amfresh
dantbeshtari
dantdardkidawa
dantmanjanlal
dantodbheddgadanta
iremedadi
karpuradibati
truedent
otoflour
floraking
mosiba
mosibagp
selenogp
haa
oratreat
gumtone
orafresh
aqwet
cinort
stroncip
ulceheal
mucomet
macrocide
jodin
tideol
chlodin
xyloplus
dentex
hexinatemouthwash
omgelct
hexide
caziq
cheerio
clohexads
clohexplus
dologel
fittydent
glizer
oragine
sflo
senquel
senquelad
senquelflami
stolin
stolinr
vantej
clenora
freshriv
gumaid
orasepelap
orasepla
sensicure
sensicured
sensicurek
sensowash
ampm
ampmplus
readyrinse
tantum
amlenox
curasil
pause
esaliva
orashield
sufrate
triguard
denoseal
denosealf
gumfortegel
quadrajel
dentone
laung
babool
clove
daburlalmanjan
daburred
dantobhedgadanras
irimedaditail
lavangatail
meswak
orabliss
parodontax
sensodyne
candidmouthgel
aclaim
amflor
anabel
anabelir
astrigum
clorni
elsenz
enafix
freshclor
glodent
guardor
guardorold
omnide

In [11]:
sequences = []
next_chars = []

# Loop over our data and extract pairs of sequances and next chars
for i in range(0, len(concat_names) - max_sequence_length, step_length):
    sequences.append(concat_names[i: i + max_sequence_length])
    next_chars.append(concat_names[i + max_sequence_length])

num_sequences = len(sequences)

print('Number of sequences:', num_sequences)
print('First 10 sequences and next chars:')
for i in range(10):
    print('X=[{}] y=[{}]'.replace('\n', ' ').format(sequences[i], next_chars[i]).replace('\n', ' '))

Number of sequences: 41529
First 10 sequences and next chars:
X=[domeric hydent hyd] y=[e]
X=[omeric hydent hyde] y=[n]
X=[meric hydent hyden] y=[t]
X=[eric hydent hydent] y=[k]
X=[ric hydent hydentk] y=[ ]
X=[ic hydent hydentk ] y=[k]
X=[c hydent hydentk k] y=[e]
X=[ hydent hydentk ke] y=[n]
X=[hydent hydentk ken] y=[a]
X=[ydent hydentk kena] y=[c]


In [12]:
X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
Y = np.zeros((num_sequences, num_chars), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for j, char in enumerate(sequence):
        X[i, j, char2idx[char]] = 1
        Y[i, char2idx[next_chars[i]]] = 1

print('X shape: {}'.format(X.shape))
print('Y shape: {}'.format(Y.shape))

X shape: (41529, 18, 27)
Y shape: (41529, 27)


In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(latent_dim,
               input_shape=(max_sequence_length, num_chars),
               recurrent_dropout=dropout_rate))
model.add(tf.keras.layers.Dense(units=num_chars, activation='softmax'))

optimizer = tf.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 500)               1056000   
_________________________________________________________________
dense_1 (Dense)              (None, 27)                13527     
Total params: 1,069,527
Trainable params: 1,069,527
Non-trainable params: 0
_________________________________________________________________


In [15]:
if load_model:
    model.load_weights(model_path)
else:
    start = time.time()
    print('Start training for {} epochs'.format(epochs))
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbosity)
    end = time.time()
    print('Finished training - time elapsed:', (end - start)/60, 'min')
if store_model:
    print('Storing model at:', model_path)
    model.save(model_path)

Start training for 20 epochs
Epoch 1/20
649/649 [==============================] - 224s 336ms/step - loss: 2.6370
Epoch 2/20
649/649 [==============================] - 215s 332ms/step - loss: 2.2241
Epoch 3/20
649/649 [==============================] - 221s 341ms/step - loss: 2.0792
Epoch 4/20
649/649 [==============================] - 234s 360ms/step - loss: 1.9538
Epoch 5/20
649/649 [==============================] - 226s 348ms/step - loss: 1.8568
Epoch 6/20
649/649 [==============================] - 215s 332ms/step - loss: 1.7691
Epoch 7/20
649/649 [==============================] - 204s 314ms/step - loss: 1.6990
Epoch 8/20
649/649 [==============================] - 188s 290ms/step - loss: 1.6469
Epoch 9/20
649/649 [==============================] - 200s 308ms/step - loss: 1.5902
Epoch 10/20
649/649 [==============================] - 199s 306ms/step - loss: 1.5445
Epoch 11/20
649/649 [==============================] - 194s 299ms/step - loss: 1.4937
Epoch 12/20
649/649 [=============

In [19]:
# Start sequence generation from end of the input sequence
sequence = concat_names[-(max_sequence_length - 1):] + '\n'

new_names = []
print('{} new names are being generated'.format(gen_amount))

while len(new_names) < gen_amount:
    # Vectorize sequence for prediction
    x = np.zeros((1, max_sequence_length, num_chars))
    for i, char in enumerate(sequence):
        x[0, i, char2idx[char]] = 1

    # Sample next char from predicted probabilities
    probs = model.predict(x, verbose=0)[0]
    probs /= probs.sum()
    next_idx = np.random.choice(len(probs), p=probs)
    next_char = idx2char[next_idx]
    sequence = sequence[1:] + next_char

    # New line means we have a new name
    if next_char == '\n':
        gen_name = [name for name in sequence.split('\n')][1]
        
        # Never start name with two identical chars, could probably also
        if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
            gen_name = gen_name[1:]
        
        # Discard all names that are too short
        if len(gen_name) > 2:
            # Only allow new and unique names
            if gen_name not in input_values + new_names:
                new_names.append(gen_name.capitalize())
        
        if 0 == (len(new_names) % (gen_amount/ 10)):
            print('Generated {}'.format(len(new_names)))

10 new names are being generated
Generated 1
Generated 2
Generated 3
Generated 4
Generated 5
Generated 6
Generated 7
Generated 8
Generated 9
Generated 10


In [21]:
print_first_n = min(10, gen_amount)
print('First {} generated names:'.format(print_first_n))
for name in new_names:
    print(name)  

First 10 generated names:
Naxdom
Mobeol
Rocunium
Asori
Astar
Agraban
Zaine
Tolrest
Cafemostopical
Tologrel


In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())